# Creating a Custom Dataset for training YOLOv8

In [51]:
import yaml
import numpy as np
import pandas as pd
import os
import cv2

### Get File Names of geom.yml and types.yml annotation files from train and validate folders
* `diva_train_path`
    - `train_files`
* `diva_valid_path`
    - `valid_files`

### Some Data Cleaning:
#### some files in the 'annotations' don't actually correspond to a VIRAT_Ground_Dataset .mp4 file

In [75]:
# get diva annotation files from training folder
diva_train_path = '/Users/p/Documents/Code/VIRAT/VIRAT_Ground_Dataset/diva_annotations/train'
train_dir = os.listdir(diva_train_path)
keep_these = {".geom", "types"}
train_geom_types = sorted([item for item in train_dir if item[-9:-4] in keep_these])
train_files = list(zip(train_geom_types[::2], train_geom_types[1::2]))
train_files

[('VIRAT_S_000000.geom.yml', 'VIRAT_S_000000.types.yml'),
 ('VIRAT_S_000001.geom.yml', 'VIRAT_S_000001.types.yml'),
 ('VIRAT_S_000002.geom.yml', 'VIRAT_S_000002.types.yml'),
 ('VIRAT_S_000005.geom.yml', 'VIRAT_S_000005.types.yml'),
 ('VIRAT_S_000200_03_000657_000899.geom.yml',
  'VIRAT_S_000200_03_000657_000899.types.yml'),
 ('VIRAT_S_000200_05_001525_001575.geom.yml',
  'VIRAT_S_000200_05_001525_001575.types.yml'),
 ('VIRAT_S_000201_03_000640_000672.geom.yml',
  'VIRAT_S_000201_03_000640_000672.types.yml'),
 ('VIRAT_S_000201_05_001081_001215.geom.yml',
  'VIRAT_S_000201_05_001081_001215.types.yml'),
 ('VIRAT_S_000201_06_001354_001397.geom.yml',
  'VIRAT_S_000201_06_001354_001397.types.yml'),
 ('VIRAT_S_000201_07_001485_001581.geom.yml',
  'VIRAT_S_000201_07_001485_001581.types.yml'),
 ('VIRAT_S_000201_08_001652_001838.geom.yml',
  'VIRAT_S_000201_08_001652_001838.types.yml'),
 ('VIRAT_S_000202_02_001527_001560.geom.yml',
  'VIRAT_S_000202_02_001527_001560.types.yml'),
 ('VIRAT_S_00020

In [81]:
annotations_list = [item[0].split('.')[0] for item in train_files]
videos_list_ = sorted(os.listdir('/Users/p/Documents/Code/VIRAT/VIRAT_Ground_Dataset/videos_original'))
videos_list = [item.split('.')[0] for item in videos_list_]

In [84]:
len(annotations_list)

64

In [85]:
len(videos_list)

329

In [87]:
dont_exist = []
for i in range(len(annotations_list)):
    if annotations_list[i] not in videos_list:
        dont_exist.append(annotations_list[i])
dont_exist

['VIRAT_S_000000',
 'VIRAT_S_000005',
 'VIRAT_S_000202_02_001527_001560',
 'VIRAT_S_000202_03_001577_001611',
 'VIRAT_S_000202_04_001704_001746',
 'VIRAT_S_000204_00_000000_000109',
 'VIRAT_S_000204_03_000667_000703',
 'VIRAT_S_000204_04_000738_000977',
 'VIRAT_S_000204_05_001334_001520',
 'VIRAT_S_000205_07_001761_001804',
 'VIRAT_S_000206_03_000500_000556',
 'VIRAT_S_000206_04_000710_000779',
 'VIRAT_S_040100_01_000347_000432',
 'VIRAT_S_040101_02_000224_000440',
 'VIRAT_S_040101_03_000460_000551',
 'VIRAT_S_040101_04_000562_000679',
 'VIRAT_S_040102_08_000986_001090',
 'VIRAT_S_040102_10_001258_001408',
 'VIRAT_S_040102_13_001557_001589']

In [88]:
remove_these = [(item+'.geom.yml', item+'.types.yml') for item in dont_exist]

In [89]:
len(train_files)

64

In [90]:
for i in range(len(remove_these)):
    train_files.remove(remove_these[i])

In [91]:
len(train_files)

45

In [95]:
# get diva annotation files from validation folder
diva_validate_path = '/Users/p/Documents/Code/VIRAT/VIRAT_Ground_Dataset/diva_annotations/validate'
valid_dir = os.listdir(diva_validate_path)
keep_these = {".geom", "types"}
valid_geom_types = sorted([item for item in valid_dir if item[-9:-4] in keep_these])
valid_files = list(zip(valid_geom_types[::2], valid_geom_types[1::2]))
valid_files

[('VIRAT_S_000007.geom.yml', 'VIRAT_S_000007.types.yml'),
 ('VIRAT_S_000008.geom.yml', 'VIRAT_S_000008.types.yml'),
 ('VIRAT_S_000200_00_000100_000171.geom.yml',
  'VIRAT_S_000200_00_000100_000171.types.yml'),
 ('VIRAT_S_000200_02_000479_000635.geom.yml',
  'VIRAT_S_000200_02_000479_000635.types.yml'),
 ('VIRAT_S_000201_00_000018_000380.geom.yml',
  'VIRAT_S_000201_00_000018_000380.types.yml'),
 ('VIRAT_S_000201_01_000384_000589.geom.yml',
  'VIRAT_S_000201_01_000384_000589.types.yml'),
 ('VIRAT_S_000201_02_000590_000623.geom.yml',
  'VIRAT_S_000201_02_000590_000623.types.yml'),
 ('VIRAT_S_000201_04_000682_000822.geom.yml',
  'VIRAT_S_000201_04_000682_000822.types.yml'),
 ('VIRAT_S_000203_01_000171_000345.geom.yml',
  'VIRAT_S_000203_01_000171_000345.types.yml'),
 ('VIRAT_S_000203_08_001702_001734.geom.yml',
  'VIRAT_S_000203_08_001702_001734.types.yml'),
 ('VIRAT_S_000204_07_001577_001611.geom.yml',
  'VIRAT_S_000204_07_001577_001611.types.yml'),
 ('VIRAT_S_000204_09_001768_001849.geo

In [97]:
annotations_list = [item[0].split('.')[0] for item in valid_files]
videos_list_ = sorted(os.listdir('/Users/p/Documents/Code/VIRAT/VIRAT_Ground_Dataset/videos_original'))
videos_list = [item.split('.')[0] for item in videos_list_]

In [98]:
dont_exist = []
for i in range(len(annotations_list)):
    if annotations_list[i] not in videos_list:
        dont_exist.append(annotations_list[i])
dont_exist

['VIRAT_S_000007',
 'VIRAT_S_000008',
 'VIRAT_S_000203_01_000171_000345',
 'VIRAT_S_000204_07_001577_001611',
 'VIRAT_S_000204_09_001768_001849',
 'VIRAT_S_000205_06_001566_001631',
 'VIRAT_S_000206_00_000025_000058',
 'VIRAT_S_000206_01_000148_000184',
 'VIRAT_S_000206_02_000294_000327',
 'VIRAT_S_040100_03_000496_000559',
 'VIRAT_S_040100_04_000626_000689',
 'VIRAT_S_040100_05_000696_000762',
 'VIRAT_S_040100_06_000767_000988',
 'VIRAT_S_040100_07_001043_001099',
 'VIRAT_S_040100_08_001103_001181',
 'VIRAT_S_040100_09_001186_001533',
 'VIRAT_S_040101_05_000722_001547',
 'VIRAT_S_040101_06_001557_001590',
 'VIRAT_S_040102_04_000596_000689',
 'VIRAT_S_040102_06_000849_000908',
 'VIRAT_S_040102_07_000916_000983']

In [99]:
len(valid_files)

55

In [101]:
remove_these = [(item+'.geom.yml', item+'.types.yml') for item in dont_exist]

In [102]:
for i in range(len(remove_these)):
    valid_files.remove(remove_these[i])

In [103]:
len(valid_files)

34

### Take a sample of the Train and Validate files for data extraction
- 30 from Train
- 10 from Validate

In [104]:
import random
random.seed(40)

train_sample = sorted(random.sample(train_files, 30))
valid_sample = sorted(random.sample(valid_files, 10))

In [105]:
len(train_sample)

30

In [106]:
len(valid_sample)

10

### Match Annotation Files to Video Files
* `train_sample` <--> `train_files`
* `valid_sample` <--> `valid_files`

In [107]:
train_files = []

for i in range(len(train_sample)):
    this_file = train_sample[i][0].split('.')[0]
    this_file += '.mp4'
    train_files.append((train_sample[i][0], train_sample[i][1], this_file))
len(train_files)

30

In [108]:
for i in range(3):
    print(train_files[27][i])

VIRAT_S_040104_06_001121_001241.geom.yml
VIRAT_S_040104_06_001121_001241.types.yml
VIRAT_S_040104_06_001121_001241.mp4


In [109]:
valid_files = []

for i in range(len(valid_sample)):
    this_file = valid_sample[i][0].split('.')[0]
    this_file += '.mp4'
    valid_files.append((valid_sample[i][0], valid_sample[i][1], this_file))
len(valid_files)

10

In [110]:
for i in range(3):
    print(valid_files[7][i])

VIRAT_S_040103_00_000000_000120.geom.yml
VIRAT_S_040103_00_000000_000120.types.yml
VIRAT_S_040103_00_000000_000120.mp4


### Get 200 images of 'Persons' with ground truth from each video.
#### Training

1. Import geom.yml and types.yml files
2. get dataframe from (1)
3. Filter for 'Persons' and create sample of frames
4. Write frames as .jpgs in Custom_Dataset/train/images/ folder with '[file-prefix]_[frame-num].jpg' as file name
5. Write .txt files by Querying full dataframe per frame for detections' labels and bboxes. write to Custom_Dataset/train/labels/ folder with '[file-prefix]_[frame-num].txt' as file name
6. Repeat 1-5 for Validation Set

In [111]:
diva_train_path = '/Users/p/Documents/Code/VIRAT/VIRAT_Ground_Dataset/diva_annotations/train'

def get_geom_types(files_pos):
    '''
    Reads .yml files into python dictionaries...
    takes a while...
    '''
    
    # geom
    geom_path = diva_train_path + '/' + train_files[files_pos][0]
    with open(geom_path, 'r') as file:
        geom = yaml.safe_load(file)

    # types
    types_path = diva_train_path + '/' + train_files[files_pos][1]
    with open(types_path, 'r') as file:
        types = yaml.safe_load(file)
    
    return geom, types

In [112]:
# Turns the geom.yml and types.yml files into a single dataframe

def get_df(geom, types):
    '''
    returns a dataframe from the geom and types dictionaries
    '''
    id0 = []  # unique detection index
    id1 = []  # track_id
    ts0 = []  # frame
    labels = []  # string class labels
    labels_ints = []  # integer class labels
    confs = []
    xmin, ymin, xmax, ymax = [], [], [], []

    for i in geom:
      try:
        if i['geom']['ts0'] is not None:  # populates detections data from geom file
          id0.append(i['geom']['id0'])
          id1.append(i['geom']['id1'])
          ts0.append(i['geom']['ts0'])
          bb = i['geom']['g0'].split(' ')
          xmin.append(int(bb[0]))
          ymin.append(int(bb[1]))
          xmax.append(int(bb[2]))
          ymax.append(int(bb[3]))

          for j in types:
            try:
              if j['types']['id1'] == id1[-1]:  # pulls labels and confidences (by track id --> 'id1') from types.yaml file
                label, conf = next(iter(j['types']['cset3'].items()))
                confs.append(conf)
                labels.append(label)
                # also save labels as ints for comparison with YOLOv8 Predictions later on
                if label == 'Person':
                  labels_ints.append(0)
                elif label == 'Bike':
                  labels_ints.append(1)
                elif label == 'Vehicle':
                  labels_ints.append(2)
                else:
                  labels_ints.append(-1)
            except:
              pass
      except:
        pass

    gt = {'idx_gt': id0, 'track_id_gt': id1, 'label_gt': labels,
          'label_as_int_gt': labels_ints, 'conf_gt': confs, 'frame_gt': ts0,
          'xmin_gt': xmin, 'ymin_gt': ymin, 'xmax_gt': xmax, 'ymax_gt': ymax}
    df = pd.DataFrame(gt)
    
    print('df is created!')
    return df

In [118]:
# sample and filter the df
def filter_and_sample(df):
    '''filters and samples from a dataframe
    
    returns: list of frame numbers
    '''
    # filter for cars and people
    df_persons = df[df['label_as_int_gt'] == 0]
    # sample of the df
    df_persons_200 = df_persons.sample(n=min(len(df_persons), 200))
    persons_frames = sorted(df_persons_200['frame_gt'].to_list())
    print('len(persons_frames): ', len(persons_frames))
    
    return persons_frames

In [114]:
def convert_bbox_format(bbox, total_width, total_height):
    """
    Converts bounding box from [xmin, ymin, xmax, ymax] format to 
    normalized [x_mid, y_mid, width, height] format.

    Parameters:
    bbox (list): A list of 4 floats: [xmin, ymin, xmax, ymax].
    total_width (int): The total width of the encoded frame.
    total_height (int): The total height of the encoded frame.

    Returns:
    list: A list of 4 floats: [x_mid, y_mid, width, height] where these values 
    have been normalized with respect to total_width and total_height.
    """
    xmin, ymin, xmax, ymax = bbox
    width = xmax - xmin
    height = ymax - ymin
    x_mid = xmin + width / 2
    y_mid = ymin + height / 2
    
    # Normalize the coordinates and dimensions
    x_mid /= total_width
    y_mid /= total_height
    width /= total_width
    height /= total_height
    
    return [x_mid, y_mid, width, height]

In [119]:
def write_files(files_pos, persons_frames, df):

    # write .jpgs and .txts for all frames of train_files[files_pos][3]
    # to '/Users/p/Documents/Code/VIRAT/VIRAT_Ground_Dataset/Custom/train/[images, labels]'
    vid_path = '/Users/p/Documents/Code/VIRAT/VIRAT_Ground_Dataset/videos_original/'
    this_vid_path = vid_path + train_files[files_pos][2]

    # begin video capture
    cap = cv2.VideoCapture(this_vid_path)

    total_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    total_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

    for i in persons_frames:
        # write .txt files:
        # get persons in frame
        persons_in_frame_df = df[(df['frame_gt'] == i) & (df['label_as_int_gt'] == 0)]

        write_this = ""
        for j in range(len(persons_in_frame_df)):
            this_person = persons_in_frame_df.iloc[j]
            label = this_person['label_as_int_gt']
            xmin = float(this_person['xmin_gt'])
            ymin = float(this_person['ymin_gt'])
            xmax = float(this_person['xmax_gt'])
            ymax = float(this_person['ymax_gt'])
            x_mid, y_mid, width, height = convert_bbox_format([xmin,ymin,xmax,ymax], total_width, total_height)

            # create string for .txt file
            write_this += str(label) + ' ' + str(x_mid) + ' ' + str(y_mid) + ' ' + str(width) + ' ' + str(height)
            if j != (len(persons_in_frame_df) - 1):
                write_this += '\n'

        # write the .txt file
        labels_loc = '/Users/p/Documents/Code/VIRAT/VIRAT_Ground_Dataset/Custom/train/labels/'
        txt_path = labels_loc + train_files[files_pos][2].split('.')[0] + '_' + str(i) + '.txt'
        with open(txt_path, 'w') as f:
            f.write(write_this)

        # write .jpg files:
        cap.set(cv2.CAP_PROP_POS_FRAMES, p)  # set frame position
        success, frame = cap.read()  # read frame
        if not success:
            print("Failed to read the frame")
        if success:
            images_loc = '/Users/p/Documents/Code/VIRAT/VIRAT_Ground_Dataset/Custom/train/images/'
            filepath = images_loc + train_files[files_pos][2].split('.')[0] + '_' + str(i) + '.jpg'
            cv2.imwrite(filepath, frame)  # write frame to .jpg and store in parent folder

    # end capture
    cap.release()

In [116]:
%%time
for files_pos in range(len(train_files)):
    geom, types = get_geom_types(files_pos)
    df = get_df(geom, types)
    persons_frames = filter_and_sample(df)
    write_files(files_pos, persons_frames, df)

df is created!
len(persons_frames):  200
df is created!
len(persons_frames):  200
df is created!
len(persons_frames):  200
df is created!
len(persons_frames):  200
df is created!
len(persons_frames):  200
df is created!
len(persons_frames):  200
df is created!
len(persons_frames):  200
df is created!
len(persons_frames):  200
df is created!
len(persons_frames):  200
df is created!
len(persons_frames):  200
df is created!
len(persons_frames):  200
df is created!
len(persons_frames):  200
df is created!
len(persons_frames):  200
df is created!
len(persons_frames):  200
df is created!
len(persons_frames):  200
df is created!


ValueError: a must be greater than 0 unless no samples are taken

In [121]:
for j in []:
    print()

In [125]:
md = {1: ['2', 'tw'], 2: [0.1, 3.3], 3:['3', 'll']}
df_qq = pd.DataFrame(data=md)

In [128]:
df_qq.drop(index=df_qq.index, inplace=True)

In [131]:
pt = sorted(df_qq[1].to_list())

In [132]:
pt

[]